Notebook to help create Dataverse table shortcut from existing Fabric Link created Lakehouse to another Lakehouse that is in the same workspace or another workspace for selected list of tables. 

In [ ]:
import requests
import json
from requests.exceptions import HTTPError

# Fabric REST API URI
base_url = "https://api.fabric.microsoft.com"

# Tables you want to filter
allowed_tables = ['ecoresconfiguration','logisticselectronicaddress','ecoresproducttranslation','logisticspostaladdress','logisticselectronicaddress','pccomponentinstancevalue','logisticspostaladdress','purchline','ecorescategoryinstancevalue','whsworkcreatehistory','purchtable','inventjournaltable','whsworkexception','reqtrans','generaljournalaccountentry','whsworkclustertable','retailcatalogprodinternalorginstanceval','ecorescurrencyvalue','whsworker','retailchannelinstancevalue','ecoresattributevalue','whsworkclusterline','retailinternalorgproductinstancevalue','ecoresproductvariantsize','whsworkline','retailpricingsimulatorinstancevalue','ecoresproductinstancevalue','whsworkusersessionlog','retailsalestableinstancevalue','ecoresdatetimevalue','whsworkblockingreasonassociation','salesline','ecoresproductvariantstyle','whsworkpool','salesline_w','ecoresfloatvalue','whsworktemplateline','salestable','gupretailchannelinstancevalue','whsworkuser','taxtrans','whsworkpolicylocation','tmsloadbuildstrategyattribvalueset','whsworktypecustom','vendgroup','whsworkuseraccesspolicy','vendtrans','ecoresproductvariantconfiguration','whsworkuserwarehouse','whsinventenabled','whsworkpolicyproduct','whsinventreserve','whsworkclasstable','whsreservationhierarchyelement','whsworklinecyclecount','whsreservationhierarchyitem','whsworktable','whsworkinventtrans','whsinventenabled','whsworktemplatetable','whsinventreserve','whsworkcreationrule','whsreservationhierarchyitem','whsworkexceptionlog','whsreservationhierarchyelement','whsworkpolicy','whsworktrans','inventtrans','ecoresinstancevalue','salestable','inventtransposting','ecoresattributetype','inventsettlement','catalogproductinstancevalue','commissionsalesgroup','custgroup','ecoresintvalue','custinvoicejour','custinvoiceline','ecoresproduct','custinvoicetable','reqtrans','custinvoicetrans','customerinstancevalue','custpackingslipjour','custpackingslipjour','custpackingsliptrans','custtable','custtable','ledgerjournaltrans','custtrans','ecoresproductvariantcolor','dirpartytable','tmsloadbuildstrategyattribvalueset','ecoresattribute','ecoresstyle','ecoresattributetype','retailpricingsimulatorinstancevalue','ecoresattributevalue','dirpartytable','ecoresbooleanvalue','ecoresattribute','ecorescategoryinstancevalue','ecoresreferencevalue','ecorescolor','custinvoicetrans','ecoresconfiguration','purchline','ecorescurrencyvalue','ecoresproductvariantdimensionvalue','ecoresdatetimevalue','generaljournalentry','ecoresdistinctproduct','vendtrans','ecoresdistinctproductvariant','custinvoiceline','ecoresengineeringproductcategoryattributeinstancevalue','retailinternalorgproductinstancevalue','retailsalestableinstancevalue','ecoresfloatvalue','guppricetreeinstancevalue','inventsum','ecoresinstancevalue','ecoresintvalue','retailcatalogprodinternalorginstanceval','inventtransorigin','ecoresproduct','ecoresdistinctproduct','inventtransoriginjournaltrans','ecoresproductinstancevalue','ecoresproductmaster','inventtransoriginhistory','ecoresproductmaster','ecoresdistinctproductvariant','inventtransoriginquarantineorder','ecoresproducttranslation','customerinstancevalue','inventtransorigintransferreceive','ecoresproductvariantcolor','pccomponentinstancevalue','inventtransoriginjournaltransreceipt','ecoresproductvariantconfiguration','salesline','inventtransoriginsalesline','ecoresproductvariantdimensionvalue','commissionsalesgroup','inventtransoriginprodbom','ecoresproductvariantsize','inventtransoriginpurchline','ecoresproductvariantstyle','ecorescolor','inventcountjour','ecoresreferencevalue','inventdimcombination','inventtransorigintransfer','ecoresreleasedengineeringproductversionattributeinstancevalue','ecoressize','inventtransoriginprodtable','ecoressize','purchtable','inventtransorigintransfertransitto','ecoresstyle','custgroup','inventtransoriginsalesquotationline','ecorestextvalue','inventjournaltrans','inventjournalname','ecoresvalue','ecoresvalue','inventtransorigintransfertransitfrom','generaljournalaccountentry','inventtransoriginblockingissue','generaljournalentry','ledgerjournaltable','inventtransorigintransfership','guppricetreeinstancevalue','custinvoicetable','ecoresreleasedengineeringproductversionattributeinstancevalue','inventtransoriginblockingreceipt','guprebatedateinstancevalue','catalogproductinstancevalue','ecorestextvalue','gupretailchannelinstancevalue','gupsalesquotationinstancevalue','gupsalestableinstancevalue','gupsalesquotationinstancevalue','vendgroup','inventdim','gupsalestableinstancevalue','retailchannelinstancevalue','hcmworker','inventdim','inventdimcombination','custpackingsliptrans','inventjournaltable','guprebatedateinstancevalue','inventjournaltrans','custtrans','inventsettlement','ecoresbooleanvalue','inventsum','salesline_w','inventtable','custinvoicejour','inventtrans','inventtable','inventtransposting','taxtrans','ledgerjournaltable','ecoresengineeringproductcategoryattributeinstancevalue','ledgerjournaltrans']

# Source WorkspaceId and lakehouse name 
source_workspace_id = "6b887591-99a7-4544-a8c4-80d4a784d7b1" 
source_lakehouse_name = 'Contoso_D365_LH'
# Target WorkspaceId and Lakehouse name 
target_workspace_id = source_workspace_id
target_lakehouse_name = 'FnO_Silver'

# update schema target_lakehouse_schema if Lakehouse is schema enabled else None
#target_lakehouse_schema = None
target_lakehouse_schema = 'dbo'

# Get lakehouse details
source_lakehouse = notebookutils.lakehouse.get(source_lakehouse_name,source_workspace_id)
source_workspace_id = source_lakehouse.workspaceId
source_lakehouse_id = source_lakehouse.id

# build source item url  
source_items_url = f"/v1/workspaces/{source_workspace_id}/items/{source_lakehouse_id}"

target_lakehouse = notebookutils.lakehouse.get(target_lakehouse_name,target_workspace_id)
target_workspace_id = target_lakehouse.workspaceId
target_lakehouse_id = target_lakehouse.id


target_items_url = f"/v1/workspaces/{target_workspace_id}/items/{target_lakehouse_id}"

# Create HTTP client session
token = notebookutils.credentials.getToken(base_url)
headers = {"Authorization": f"Bearer {token}", "Content-Type": "application/json"}
client = requests.Session()

# Get all tables
artifacts_tables_list = notebookutils.lakehouse.listTables(source_lakehouse_name, source_workspace_id)

# Filter by allowed table names (using dict keys, not attributes)
filtered_tables = [table for table in artifacts_tables_list if table['name'] in allowed_tables]

for table in filtered_tables:
    table_name = table['name']

    # get table shortcut payload from source
    source_url= base_url + f"{source_items_url}/shortcuts/Tables/{table_name}"
    print(f"Processing table: {table_name}")
    response = client.get(source_url, headers=headers)
    shortcut_payload = response.json()  

    if (target_lakehouse_schema is not None):
    # override the schema if present for target
        shortcut_payload['path'] = f"Tables/{target_lakehouse_schema}"
    
    payload_json = json.dumps(shortcut_payload, indent=4)
    #print(f"Payload for table '{table_name}': {payload_json}")

    try:
        response = client.post(
            base_url + f"{target_items_url}/shortcuts?shortcutConflictPolicy=Abort",
            data=payload_json,
            headers=headers
        )
        response.raise_for_status()
        print(f"Shortcut created successfully for table: {table_name}")
    except HTTPError as http_err:
        if ("409" in str(http_err)):  
            # Conflict error, e.g., if the shortcut already exists:
            print(f"Shortcut already exists for table: {table_name}")
        else:
            print(f"HTTP error occurred for table '{table_name}': {http_err}")
    except Exception as err:
        print(f"Unexpected error occurred for table '{table_name}': {err}")
    else:
        print(f"Response: {json.dumps(response.json(), indent=4)}")
